In [2]:
import numpy as np

def sample_numerical_kform_neighborhood_val(sampler, point, epsilon=1e-8):
    """
    Sample numerical values of a k-form at a (cubic) neighborhood of a point
    :param sampler: function to sample the k-form
    :param point: point at which to sample (center of a cube)
    :param epsilon: distance from the point to the neighborhood points
    :return: numerical values of the k-form at the neighborhood points as a dictionary
    6-dimensional dictionary with keys '0' to '6', where '0' is the center point and '1' to '6' are the -+ epsilon* canonical basis vectors 
    """
    p = np.array(point)
    directions = [np.array(np.eye(6, dtype=int)[i]) for i in range(6)]
    values = {"0": [sampler(p)],
            "1": [sampler(p - epsilon*directions[0]), sampler(p + epsilon*directions[0])],
            "2": [sampler(p - epsilon*directions[1]), sampler(p + epsilon*directions[1])],
            "3": [sampler(p - epsilon*directions[2]), sampler(p + epsilon*directions[2])],
            "4": [sampler(p - epsilon*directions[3]), sampler(p + epsilon*directions[3])],
            "5": [sampler(p - epsilon*directions[4]), sampler(p + epsilon*directions[4])],
            "6": [sampler(p - epsilon*directions[5]), sampler(p + epsilon*directions[5])],
            
            } 
            
    return values
    
def numerical_d(k_form_neighborhood_vals, e=1e-8):
    # d\omega_p(X_0, ..., X_k) \approx \frac{1}{\text{vol}(\sigma)} \int_{\partial \sigma} \omega 
    # where \sigma is a small k-dimensional block generated by X_0, ..., X_k
    # ----
    # Also, remember g_{ij}(x) = \delta_{ij} - \frac{1}{3} R_{ikjl}(0) \, x^k x^l + \mathcal{O}(|x|^3) = 
    #                          =\delta_{ij} + \mathcal{O}(|x|^3), 
    # since R_{ikjl} = 0 for Ricci-flat manifolds (which we assume here)
    # Thus, the volume of the k-dimensional block is approximately (2e)^k 
    # ----
    # Note for future Tomas: (-1)xSign convention of Acker's Book!
    """
    Compute the numerical exterior derivative of a k-form at a point using neighborhood values
    :param k_form_neighborhood_vals: dictionary of neighborhood values of the k-form
    :param g: metric tensor (needed for computing hypercube volume)
    :param e: small value to approximate the derivative
    :return: numerical exterior derivative of the k-form a the point k_form_neighborhood_vals["0"] as a numpy array
    """
    
    k = len(k_form_neighborhood_vals["0"][0].shape)
    
    # If the k-form is a 0-form (scalar field)
    if k == 0: 
        dw = np.zeros(6)
        for i in range(6):
            dw[i] = (1/(2*e))*(k_form_neighborhood_vals[str(i+1)][1] - k_form_neighborhood_vals[str(i+1)][0])
        return dw
    
    # If the k-form is a 1-form
    elif k == 1:
        dw = np.zeros((6,6))
        for i in range(6):
            for j in range(6):
                if i!=j:
                    dw[i,j] = (1/(2*e))*(k_form_neighborhood_vals[str(j+1)][0][i] +\
                                                k_form_neighborhood_vals[str(i+1)][1][j] -\
                                                k_form_neighborhood_vals[str(j+1)][1][i] -\
                                                k_form_neighborhood_vals[str(i+1)][0][j])
        
        return dw
    
    # If the k-form is a 2-form
    elif k == 2:
        dw = np.zeros((6,6,6))      
        for i in range(6):
            for j in range(6):
                for k in range(6):
                    if i!=j and j!=k and i!=k:
                        dw[i,j,k] = (1/(2*e))*(+k_form_neighborhood_vals[str(i+1)][1][j,k] -\
                                                k_form_neighborhood_vals[str(i+1)][0][j,k] -\
                                                k_form_neighborhood_vals[str(j+1)][1][i,k] +\
                                                k_form_neighborhood_vals[str(j+1)][0][i,k] +\
                                                k_form_neighborhood_vals[str(k+1)][1][i,j] -\
                                                k_form_neighborhood_vals[str(k+1)][0][i,j])
        return dw

    elif k >= 3:
        raise NotImplementedError("Numerical exterior derivative for k-forms with k >= 3 is not implemented.")

In [22]:
# The actual sampler will be a trained model that gets the kahler form at any point!!!
def sampler_2form(_):
    """
    Dummy sampler function for testing purposes.
    Replace this with the actual sampling function for your k-form.
    """
    return np.random.rand(6,6)  # Example: returns a random 6-dimensional vector

In [23]:
dic2_form = sample_numerical_kform_neighborhood_val(sampler_2form, [0, 0, 0, 0, 0, 0])
dic2_form

{'0': [array([[0.38786111, 0.33332299, 0.660848  , 0.78068774, 0.14329412,
          0.72727982],
         [0.38527355, 0.82225351, 0.37937936, 0.49120837, 0.24233726,
          0.46418403],
         [0.68022689, 0.2202353 , 0.24768433, 0.52294378, 0.22800793,
          0.79078605],
         [0.88354916, 0.68959588, 0.05711858, 0.54360761, 0.45470453,
          0.49238948],
         [0.35098213, 0.69109662, 0.48836745, 0.22056304, 0.22028746,
          0.48233073],
         [0.52420547, 0.39109624, 0.32654378, 0.54575247, 0.7691665 ,
          0.42745101]])],
 '1': [array([[0.56955814, 0.26509545, 0.09014735, 0.41512981, 0.34171019,
          0.26394174],
         [0.99138064, 0.86320833, 0.82179763, 0.31706824, 0.15964848,
          0.68905502],
         [0.11319536, 0.55483723, 0.0125041 , 0.6869238 , 0.23985844,
          0.55902472],
         [0.67690799, 0.03572198, 0.44766489, 0.7313505 , 0.74986066,
          0.25307689],
         [0.17611986, 0.56397782, 0.77980661, 0.38045244,

In [24]:
# NB: you should pass a dict for the numerical_d function, sice it expects values in the neighborhood of the point
numerical_exterior_d.shape #Returns (6, 6, 6), a 3-form at the point (0, 0, 0, 0, 0, 0)

(6, 6, 6)